In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1
%env PYTORCH_CUDA_ALLOC_CONF backend:cudaMallocAsync

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
dataset = 'mp_ternary'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=False, dataset=dataset)
print(max_len)

21


In [3]:
del torch_datasets['test'], datasets_pd['test']

In [4]:
from itertools import chain
from operator import itemgetter
max_enumeration = max(map(
    lambda dataframe: max(chain.from_iterable(dataframe.symmetry_sites_enumeration_padded)),
    datasets_pd.values()))
print(max_enumeration)

7


In [5]:
import torch
# UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
torch.set_float32_matmul_precision('high')
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 170  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 3  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 1  # number of heads in ``nn.MultiheadAttention``
dropout = 0.1  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    max_enumeration = max_enumeration,
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    max_len=max_len,
    nlayers=nlayers,
    dropout=dropout).to(device)
#model = torch.compile(model)

In [6]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len,
                         site_mask=torch.tensor(site_to_ids[MASK_SITE]).to(device),
                         element_pad=torch.tensor(element_to_ids["PAD"]).to(device),
                         site_pad=torch.tensor(site_to_ids["PAD"]).to(device))
trainer.train(epochs=10000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


Epoch 10 val_loss_epoch 43.08405303955078 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 50 val_loss_epoch 42.17355728149414 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 60 val_loss_epoch 33.82870864868164 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 90 val_loss_epoch 31.943235397338867 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 120 val_loss_epoch 27.524681091308594 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 150 val_loss_epoch 27.292049407958984 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 170 val_loss_epoch 26.128114700317383 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 190 val_loss_epoch 25.279027938842773 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 250 val_loss_epoch 25.17259407043457 saved to checkpoints/2024-05-14_15-37-30/best_model_params.pt
Epoch 270 val_loss_epoch 22.729679107666016 saved to c